In [21]:
from smplx.lbs import batch_rodrigues, batch_rigid_transform
from smplx import SMPLX
import numpy as np
import torch


In [ ]:
from smplx.lbs import batch_rodrigues, batch_rigid_transform
from smplx import SMPLX
import numpy as np
import torch

# 1. 加载数据
data = np.load('../retarget_input/B1_-_stand_to_walk_stageii.npz')
pose_body = data['pose_body']           # (N, 63)  21 joints for body pose
root_orient = data['root_orient']       # (N, 3)
betas = data['betas'][:10]                   
N = pose_body.shape[0]          # frame cnt
betas = np.tile(betas[None, :], (N, 1))  # (N, num_betas)


print(pose_body.shape)
print(root_orient.shape)
print(betas.shape)
print(N)

(747, 63)
(747, 3)
(747, 10)
747


In [23]:
model = SMPLX(model_path='../human_model/smplx/SMPLX_FEMALE.npz', gender='female', batch_size=N)

In [ ]:
N = pose_body.shape[0]

# 2. full pose
full_pose = np.concatenate([root_orient, pose_body], axis=1)  # (N, 66)

# 3. rot matrix
pose_rotmats = batch_rodrigues(torch.tensor(full_pose.reshape(-1, 3), dtype=torch.float32))  # (N * 22, 3, 3)
pose_rotmats = pose_rotmats.reshape(N, 22, 3, 3)  # (N, 22, 3, 3)

output = model(
    betas=torch.tensor(betas, dtype=torch.float32),
    body_pose=torch.tensor(pose_body, dtype=torch.float32),
    global_orient=torch.tensor(root_orient, dtype=torch.float32),
    return_verts=False
)
joints = output.joints[:, :22]  # (N, 22, 3)

# 5. kinematic tree
parents = model.parents[:22].tolist()
parents = torch.tensor(parents)
print(parents)

# 6. get transform
_, transforms = batch_rigid_transform(pose_rotmats, joints, parents)  # (N, 22, 4, 4)
transforms

tensor([-1,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  9, 12, 13, 14,
        16, 17, 18, 19])


tensor([[[[-0.9997, -0.0090, -0.0216, -0.0018],
          [-0.0202, -0.1389,  0.9901, -0.4362],
          [-0.0119,  0.9903,  0.1387,  0.3793],
          [ 0.0000,  0.0000,  0.0000,  1.0000]],

         [[-0.9996, -0.0146, -0.0240, -0.0041],
          [-0.0246,  0.0403,  0.9989, -0.3669],
          [-0.0136,  0.9991, -0.0407,  0.3663],
          [ 0.0000,  0.0000,  0.0000,  1.0000]],

         [[-0.9996,  0.0074,  0.0266,  0.0091],
          [ 0.0269,  0.0516,  0.9983, -0.3665],
          [ 0.0060,  0.9986, -0.0518,  0.3629],
          [ 0.0000,  0.0000,  0.0000,  1.0000]],

         ...,

         [[-0.1118,  0.9347, -0.3373,  0.3552],
          [-0.4835,  0.2454,  0.8402, -0.1429],
          [ 0.8682,  0.2570,  0.4245, -0.0790],
          [ 0.0000,  0.0000,  0.0000,  1.0000]],

         [[ 0.0442, -0.9957,  0.0818, -0.4044],
          [ 0.5782,  0.0923,  0.8106, -0.1498],
          [-0.8147,  0.0115,  0.5798, -0.1233],
          [ 0.0000,  0.0000,  0.0000,  1.0000]],

         [[ 0.0